## LeNet-5 (MNIST)

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the LeNet-5 architecture
class LeNet5(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 6, kernel_size=5, stride=1, padding=2)  # Convolutional layer with 6 feature maps of size 5x5
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)  # Subsampling layer with 6 feature maps of size 2x2
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)  # Convolutional layer with 16 feature maps of size 5x5
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)  # Subsampling layer with 16 feature maps of size 2x2
        
        # These layers depend on the input size
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # Fully connected layer, output size 120
        self.fc2 = nn.Linear(120, 84)  # Fully connected layer, output size 84
        self.fc3 = nn.Linear(84, num_classes)  # Fully connected layer, output size num_classes

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Apply ReLU after conv1
        x = self.pool1(x)  # Apply subsampling pool1
        x = F.relu(self.conv2(x))  # Apply ReLU after conv2
        x = self.pool2(x)  # Apply subsampling pool2
        x = x.view(x.size(0), -1)  # Flatten for fully connected layers
        x = F.relu(self.fc1(x))  # Apply ReLU after fc1
        x = F.relu(self.fc2(x))  # Apply ReLU after fc2
        x = self.fc3(x)  # Output layer
        return x

# Define a function to train the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Define a function to test the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')
    
# define device
def check_gpu(manual_seed=True, print_info=True):
    if manual_seed:
        torch.manual_seed(0)
    if torch.cuda.is_available():
        if print_info:
            print("CUDA is available")
        device = 'cuda'
        torch.cuda.manual_seed_all(0) 
    elif torch.backends.mps.is_available():
        if print_info:
            print("MPS is available")
        device = torch.device("mps")
        torch.mps.manual_seed(0)
    else:
        if print_info:
            print("CUDA is not available")
        device = 'cpu'
    return device

# Main function to run the training and testing
def main():
    # Training settings
    batch_size = 64
    test_batch_size = 1000
    epochs = 10
    lr = 0.01
    momentum = 0.9
    no_cuda = False
    seed = 1

    device = check_gpu(manual_seed=True, print_info=True)

    torch.manual_seed(seed)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('../data', train=False, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

    model = LeNet5(in_channels=1, num_classes=10).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

if __name__ == '__main__':
    main()

MPS is available
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316792
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.817093
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.199695
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.175400
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.296527
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.156364
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.231877
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.090156
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.182235
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.116326

Test set: Average loss: 0.0909, Accuracy: 9724/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.138631
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.121906
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.031991
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.108236
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.121946
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.011026
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.043982
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.019277
Train Epoch: 2 [51200/60000 (85%)]

## ResNet-9 (MNIST)

In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the conv_bn_relu_pool function
def conv_bn_relu_pool(in_channels, out_channels, pool=False):
    layers = [
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    ]
    if pool:
        layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)


class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.prep = conv_bn_relu_pool(in_channels, 64)
        self.layer1_head = conv_bn_relu_pool(64, 128, pool=True)
        self.layer1_residual = nn.Sequential(conv_bn_relu_pool(128, 128), conv_bn_relu_pool(128, 128))
        self.layer2 = conv_bn_relu_pool(128, 256, pool=True)
        self.layer3_head = conv_bn_relu_pool(256, 512, pool=True)
        self.layer3_residual = nn.Sequential(conv_bn_relu_pool(512, 512), conv_bn_relu_pool(512, 512))
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Changed to adaptive average pooling:         self.MaxPool2d = nn.Sequential(nn.MaxPool2d(4))
        self.linear = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.prep(x)
        x = self.layer1_head(x)
        x = self.layer1_residual(x) + x
        x = self.layer2(x)
        x = self.layer3_head(x)
        x = self.layer3_residual(x) + x
        x = self.avgpool(x)  # Changed to adaptive average pooling
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

# Define a function to train the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Define a function to test the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

# Main function to run the training and testing
def main():
    # Training settings
    batch_size = 64
    test_batch_size = 1000
    epochs = 10
    lr = 0.01
    momentum = 0.9
    no_cuda = False
    seed = 1

    device = check_gpu(manual_seed=True, print_info=True)

    torch.manual_seed(seed)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('../data', train=False, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

    model = ResNet9(in_channels=1, num_classes=10).to(device)  # Set in_channels to 1 for MNIST
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

if __name__ == '__main__':
    main()

MPS is available
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.393180
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.125153
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.061068
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.140412
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.020400
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.041769
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.004259
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.029730
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.007122
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.084373

Test set: Average loss: 0.0489, Accuracy: 9847/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.074853
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.016679
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.052707
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.055934
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.004753
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.023614
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.115123
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.003418
Train Epoch: 2 [51200/60000 (85%)]

In [24]:
# model parameters
model_lenet = LeNet5()
print(f'LeNet-5 model has {sum(p.numel() for p in model_lenet.parameters()):,} parameters')
model_resnet9 = ResNet9(in_channels=1, num_classes=9)
print(f'ResNet-9 model has {sum(p.numel() for p in model_resnet9.parameters()):,} parameters')

LeNet-5 model has 61,706 parameters
ResNet-9 model has 6,573,705 parameters


## Trainings on ours dataset

In [33]:
import non_iiddata_generator_no_drifting as noniidgen

def data_creation(dataset_name):
    # Load the dataset
    # Options: "MNIST", "FMNIST", "CIFAR10", "CIFAR100", etc.
    train_images, train_labels, test_images, test_labels = noniidgen.load_full_datasets(dataset_name)

    # Define parameters for split_feature_skew
    client_number = 10
    set_rotation = True
    rotations = 4
    scaling_rotation_low = 0.1
    scaling_rotation_high = 0.2
    set_color = True
    colors = 3
    scaling_color_low = 0.1
    scaling_color_high = 0.2
    random_order = True

    # Run split_feature_skew
    clients_data = noniidgen.split_feature_skew(
        train_features = train_images,
        train_labels = train_labels,
        test_features = test_images,
        test_labels = test_labels,
        client_number = client_number,
        set_rotation = set_rotation,
        rotations = rotations,
        scaling_rotation_low = scaling_rotation_low,
        scaling_rotation_high = scaling_rotation_high,
        set_color = set_color,
        colors = colors,
        scaling_color_low = scaling_color_low,
        scaling_color_high = scaling_color_high,
        random_order = random_order
    )
    return clients_data

data_MNIST = data_creation("MNIST")
data_FMNIST = data_creation("FMNIST")
data_CIFAR10 = data_creation("CIFAR10")
data_CIFAR100 = data_creation("CIFAR100")

100%|██████████| 26421880/26421880 [00:01<00:00, 18718859.84it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 3326388.72it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 20956104.21it/s]


...

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 170498071/170498071 [00:19<00:00, 8610973.68it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


100%|██████████| 169001437/169001437 [00:19<00:00, 8677287.56it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [63]:
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import transforms
from math import prod

class CombinedDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.labels[idx]

        if self.transform:
            x = self.transform(x)

        return x, y
    
def merge_data(data):
    train_features = []
    train_labels = []
    test_features = []
    test_labels = []
    for client_data in data:
        train_features.append(client_data['train_features'])
        train_labels.append(client_data['train_labels'])
        test_features.append(client_data['test_features'])
        test_labels.append(client_data['test_labels'])

    # Concatenate all the data
    train_features = torch.cat(train_features, dim=0)
    train_labels = torch.cat(train_labels, dim=0)
    test_features = torch.cat(test_features, dim=0)
    test_labels = torch.cat(test_labels, dim=0)

    return train_features, train_labels, test_features, test_labels
    
class LeNet5(nn.Module):
    def __init__(self, in_channels=1, num_classes=10, input_size=(28, 28)):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 6, kernel_size=5, stride=1, padding=2)  # Convolutional layer with 6 feature maps of size 5x5
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)  # Subsampling layer with 6 feature maps of size 2x2
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)  # Convolutional layer with 16 feature maps of size 5x5
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)  # Subsampling layer with 16 feature maps of size 2x2
        
        # Calculate the size of the features after convolutional layers
        dummy_input = torch.zeros(1, in_channels, *input_size)
        dummy_output = self.pool2(self.conv2(self.pool1(self.conv1(dummy_input))))
        self.feature_size = prod(dummy_output.size()[1:])
        
        self.fc1 = nn.Linear(self.feature_size, 120)  # Fully connected layer, output size 120
        self.fc2 = nn.Linear(120, 84)  # Fully connected layer, output size 84
        self.fc3 = nn.Linear(84, num_classes)  # Fully connected layer, output size num_classes

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Apply ReLU after conv1
        x = self.pool1(x)  # Apply subsampling pool1
        x = F.relu(self.conv2(x))  # Apply ReLU after conv2
        x = self.pool2(x)  # Apply subsampling pool2
        x = x.view(x.size(0), -1)  # Flatten for fully connected layers
        x = F.relu(self.fc1(x))  # Apply ReLU after fc1
        x = F.relu(self.fc2(x))  # Apply ReLU after fc2
        x = self.fc3(x)  # Output layer
        return x

# Define a function to train the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Define a function to test the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')
    
# define device
def check_gpu(manual_seed=True, print_info=True):
    if manual_seed:
        torch.manual_seed(0)
    if torch.cuda.is_available():
        if print_info:
            print("CUDA is available")
        device = 'cuda'
        torch.cuda.manual_seed_all(0) 
    elif torch.backends.mps.is_available():
        if print_info:
            print("MPS is available")
        device = torch.device("mps")
        torch.mps.manual_seed(0)
    else:
        if print_info:
            print("CUDA is not available")
        device = 'cpu'
    return device

# Main function to run the training and testing
def main():
    # Training settings
    batch_size = 64
    test_batch_size = 1000
    epochs = 10
    lr = 0.01
    momentum = 0.9
    no_cuda = False
    seed = 1

    device = check_gpu(manual_seed=True, print_info=True)

    torch.manual_seed(seed)

    # merge the data
    train_features, train_labels, test_features, test_labels = merge_data(data_CIFAR10)

    # Define any necessary transforms
    transform = None

    # Create the datasets
    train_dataset = CombinedDataset(train_features, train_labels, transform=transform)
    test_dataset = CombinedDataset(test_features, test_labels, transform=transform)

    # Define batch sizes
    batch_size = 64
    test_batch_size = 1000

    # Create the data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

    # model = LeNet5(in_channels=3, num_classes=10, input_size=(32,32)).to(device)
    model = ResNet9(in_channels=3, num_classes=10).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

if __name__ == '__main__':
    main()

MPS is available
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.492195
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.239441
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.164107
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.944633
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.713479
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.004442
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.913772
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.762644

Test set: Average loss: 1.7305, Accuracy: 3724/10000 (37%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.895573
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.940766
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.958923
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.622462
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.713544
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.967214
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.407248
Train Epoch: 2 [44800/50000 (90%)]	Loss: 1.528836

Test set: Average loss: 1.6656, Accuracy: 4110/10000 (41%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 1.456856
Train Epoch: 3 [6400/50000 (

## Examples with our models.py

In [2]:
from models import *
import non_iiddata_generator_no_drifting as noniidgen
from non_iiddata_generator_no_drifting import merge_data
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

def main():
    # Training settings
    model_name = "LeNet5"   # Options: "LeNet5", "ResNet9"
    batch_size = 64
    test_batch_size = 1000
    epochs = 10
    lr = 0.01
    momentum = 0.9
    seed = 1
    transform = None
    # dataset settings
    dataset_name = "CIFAR10"
    client_number = 10
    set_rotation = True
    rotations = 4
    scaling_rotation_low = 0.1
    scaling_rotation_high = 0.2
    set_color = True
    colors = 3
    scaling_color_low = 0.1
    scaling_color_high = 0.2
    random_order = True

    device = check_gpu(manual_seed=True, print_info=True)
    torch.manual_seed(seed)

    # load data 
    train_images, train_labels, test_images, test_labels = noniidgen.load_full_datasets(dataset_name)

    # create data: split_feature_skew
    clients_data = noniidgen.split_feature_skew(
        train_features = train_images,
        train_labels = train_labels,
        test_features = test_images,
        test_labels = test_labels,
        client_number = client_number,
        set_rotation = set_rotation,
        rotations = rotations,
        scaling_rotation_low = scaling_rotation_low,
        scaling_rotation_high = scaling_rotation_high,
        set_color = set_color,
        colors = colors,
        scaling_color_low = scaling_color_low,
        scaling_color_high = scaling_color_high,
        random_order = random_order
    )

    # merge the data (for Centralized Learning Simulation)
    train_features, train_labels, test_features, test_labels = merge_data(clients_data)

    # Create the datasets
    train_dataset = CombinedDataset(train_features, train_labels, transform=transform)
    test_dataset = CombinedDataset(test_features, test_labels, transform=transform)

    # Create the data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

    # model = LeNet5(in_channels=3, num_classes=10, input_size=(32,32)).to(device)
    model = models[model_name](in_channels=3, num_classes=10, input_size=(32,32)).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)


    for epoch in range(1, epochs + 1):
        simple_train(model, device, train_loader, optimizer, epoch)
        simple_test(model, device, test_loader)

if __name__ == '__main__':
    main()

MPS is available
Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.311269
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.299786
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.297932
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.296077
Train Epoch: 1 [25600/50000 (51%)]	Loss: 2.280352
Train Epoch: 1 [32000/50000 (64%)]	Loss: 2.140450
Train Epoch: 1 [38400/50000 (77%)]	Loss: 2.250665
Train Epoch: 1 [44800/50000 (90%)]	Loss: 2.247782

Test set: Average loss: 2.1686, Accuracy: 2076/10000 (21%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 2.215068
Train Epoch: 2 [6400/50000 (13%)]	Loss: 2.160078
Train Epoch: 2 [12800/50000 (26%)]	Loss: 2.154256
Train Epoch: 2 [19200/50000 (38%)]	Loss: 2.040477
Train Epoch: 2 [25600/50000 (51%)]	Loss: 2.118145
Train Epoch: 2 [32000/50000 (64%)]	Loss: 2.168162
Train Epoch: 2 [38400/50000 (77%)]	Loss: 2.121017
Train Epoch: 2 [44800/50000 (90%)]	Loss: 2.161525

Test set: Average loss: 2.0218, Accuracy: 2659/10000 (27%

KeyboardInterrupt: 

In [3]:
# print the number of parameters
model_lenet = LeNet5(in_channels=3, num_classes=10, input_size=(32,32))
print(f'LeNet-5 model has {sum(p.numel() for p in model_lenet.parameters()):,} parameters')
model_resnet9 = ResNet9(in_channels=3, num_classes=10, input_size=(32,32))
print(f'ResNet-9 model has {sum(p.numel() for p in model_resnet9.parameters()):,} parameters')

model_lenet = LeNet5(in_channels=3, num_classes=10, input_size=(28,28))
print(f'LeNet-5 model has {sum(p.numel() for p in model_lenet.parameters()):,} parameters')
model_resnet9 = ResNet9(in_channels=3, num_classes=10, input_size=(28,28))
print(f'ResNet-9 model has {sum(p.numel() for p in model_resnet9.parameters()):,} parameters')



LeNet-5 model has 83,126 parameters
ResNet-9 model has 6,590,730 parameters
LeNet-5 model has 62,006 parameters
ResNet-9 model has 6,575,370 parameters


In [2]:
import torch
import models    
import config as cfg
import numpy as np
import utils
import torch.optim as optim
from torch.utils.data import DataLoader

# check gpu and set manual seed
device = utils.check_gpu(manual_seed=True)

# model and history folder
model = models.models[cfg.model_name](in_channels=3, num_classes=cfg.n_classes, input_size=cfg.input_size).to(device)
# train_fn = utils.trainings[args.model]
# evaluate_fn = utils.evaluations[args.model]
# plot_fn = utils.plot_functions[args.model]
# config = utils.config_tests[args.dataset][args.model]

# check if metrics.csv exists otherwise delete it
# utils.check_and_delete_metrics_file(config['history_folder'] + f"client_{args.data_type}_{args.id}", question=False)

# load data
data = np.load(f'./data/client_{1}.npy', allow_pickle=True).item()
num_examples = data['train_features'].shape[0]

# Create the datasets
train_dataset = models.CombinedDataset(data['train_features'], data['train_labels'], transform=None)

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)

# Optimizer and Loss function
optimizer = torch.optim.SGD(model.parameters(), lr=cfg.lr, momentum=cfg.momentum)


for epoch in range(2):
                models.simple_train(model, device, train_loader, optimizer, epoch)



MPS is available
Train Epoch: 0 [0/5000 (0%)]	Loss: 2.305545
Train Epoch: 0 [640/5000 (13%)]	Loss: 2.302148
Train Epoch: 0 [1280/5000 (25%)]	Loss: 2.309596
Train Epoch: 0 [1920/5000 (38%)]	Loss: 2.300848
Train Epoch: 0 [2560/5000 (51%)]	Loss: 2.302197
Train Epoch: 0 [3200/5000 (63%)]	Loss: 2.310703
Train Epoch: 0 [3840/5000 (76%)]	Loss: 2.304026
Train Epoch: 0 [4480/5000 (89%)]	Loss: 2.303044
Train Epoch: 1 [0/5000 (0%)]	Loss: 2.303247
Train Epoch: 1 [640/5000 (13%)]	Loss: 2.304303
Train Epoch: 1 [1280/5000 (25%)]	Loss: 2.296026
Train Epoch: 1 [1920/5000 (38%)]	Loss: 2.301974
Train Epoch: 1 [2560/5000 (51%)]	Loss: 2.303057
Train Epoch: 1 [3200/5000 (63%)]	Loss: 2.295039
Train Epoch: 1 [3840/5000 (76%)]	Loss: 2.297525
Train Epoch: 1 [4480/5000 (89%)]	Loss: 2.304764


In [4]:
from collections import OrderedDict


def get_parameters(model, config):
        return [val.cpu().numpy() for _, val in model.state_dict().items()]

def set_parameters(model, parameters):
        params_dict = zip(model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        model.load_state_dict(state_dict, strict=True)

In [5]:
x = get_parameters(model, cfg)

In [6]:
set_parameters(model, x)